In [1]:
import re
import json
import requests
import warnings
import pandas as pd
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")
%matplotlib inline

## Параметры

In [2]:
## Основной url для api запросов
main_url='https://gateway.scan-interfax.ru/'

login = 'kiselevI@open.ru'
password = 'lLd/ENl' 

## Подгрузка расшифровки
http_status_code = pd.read_csv(r'http_status_code.csv', sep=';', error_bad_lines=False, encoding="cp1251")

---------------------

# Методы работы с аккаунтом

## Авторизация

In [3]:
api_login = 'api/v1/account/login'
url = main_url + api_login
print(url)

## Параметры
params ={
  "login": login,
  "password": password
}

try:
    response = requests.post(url, json=params, verify=False)
    accessToken = response.json()['accessToken']
    print('- Результат запроса:',http_status_code[http_status_code['code']==response.status_code]['name'].values[0])
    print(http_status_code[http_status_code['code']==response.status_code]['message'].values[0])
    print('- Срок действия аккаунта до:',pd.to_datetime(response.json()['expire']).date())  
except BaseException as error:
    print('Ошибка запроса:',response.status_code)
    print(error.__doc__)

https://gateway.scan-interfax.ru/api/v1/account/login
- Результат запроса: OK
"Успешно". Запрос успешно обработан. Что значит "успешно", зависит от метода HTTP, который был запрошен:
- Срок действия аккаунта до: 2020-04-04


## Получение текущего баланса пользователя

In [4]:
api_balance = 'api/v1/account/balance'
url = main_url + api_balance
print(url)

headers = {
    'Authorization': 'Bearer ' + accessToken
}

response = requests.get(url, headers=headers, verify=False)
print('-- Результат запроса:', response.status_code)
print('-Признак безлимитного тарифа:', response.json()['unlimited'])
print('-Остаток средств на счету (у.е.):', response.json()['balance'])
print('-Стоимость одного поискового терма (у.е.):',response.json()['searchTermCost'])
print('-Время до окончания ограничения на выполнение поиска (сек.):', response.json()['searchRateBlockPeriod'])

https://gateway.scan-interfax.ru/api/v1/account/balance
-- Результат запроса: 200
-Признак безлимитного тарифа: False
-Остаток средств на счету (у.е.): 84482.0
-Стоимость одного поискового терма (у.е.): 1.0
-Время до окончания ограничения на выполнение поиска (сек.): 0


## Получение историю покупок пользователя

In [5]:
api_hist = 'api/v1/account/purchaseHistory'
url = main_url + api_hist
print(url)

headers = {
    'Authorization': 'Bearer ' + accessToken
}

response = requests.get(url, headers=headers, verify=False)
response.json()

https://gateway.scan-interfax.ru/api/v1/account/purchaseHistory


{}

# Поиск документов в каталоге

In [ ]:
# 3123027569
# 0275073194
# 5263081418
# 5752046628
# 5908048172
# 6660144200
# 7743095125
# 4727002311
# 6140005000
# 7804601371
# 3908602319
# 1102061710

In [ ]:
api_find = 'api/v1/entities/findEntities'
url = main_url + api_find
print(url)

headers = {
    'Authorization': 'Bearer ' + accessToken
}

params = {
  "matchTerm": "Газпром",
  "cardAvailable": True,
  "inn": "",
  "maxCountToReturn": 10000000
}

response = requests.post(url, json = params, headers=headers, verify=False)
print('Код запроса:', response.status_code)
print('По запрашиваемому клиенту было возвращено:',len(response.json()['entities']),'статей')
list_ent = []
# response.json()['entities'][0]['searchTerm']['mentionContext']
for ent in response.json()['entities']:
    list_ent.append({'entityId': ent['entityId'], 'entityType': ent['entityType'], 'name': ent['name'], 'searchTerm': ent['searchTerm']})

entities = pd.DataFrame(list_ent)
entities.head()

## Получение подробной информации об объекте, заданном по ID

In [ ]:
api_get_obj= 'api/v1/entities/519830'
url = main_url + api_get_obj
print(url)

headers = {
    'Authorization': 'Bearer ' + accessToken
}

response = requests.get(url, headers=headers, verify=False)
print('Код запроса:', response.status_code)
obj_info = {'image': response.json()['image'], 'description': response.json()['description'], \
            'entityId': response.json()['entityId'], 'entityType': response.json()['entityType'],
            'name': response.json()['name']}

df = pd.DataFrame(obj_info, index=[0])
df.head()

## Получение значения Индекса репутационного риска (ИРР) по заданной организации

In [ ]:
api_irr = 'api/v1/entities/getRisk'
url = main_url + api_irr
print(url)

headers = {
    'Authorization': 'Bearer ' + accessToken
}

params = {
   'entityId' : 519830
}

response = requests.post(url, json=params, headers=headers, verify=False)
print('Код запроса:', response.status_code)
res = pd.DataFrame({'value':response.json()['value'], 'riskLevel':response.json()['riskLevel'], 'documentsCount': response.json()['documentsCount'], 'documentsCountLevel':response.json()['documentsCountLevel']}, index=[0])
res.head()

## Получение списка тем публикаций

In [ ]:
api_subj= 'api/v1/entities/subjects'
url = main_url + api_subj
print(url)

headers = {
    'Authorization': 'Bearer ' + accessToken
}

params = {
    'index' : 0,
    'pageSize': 1000
}

response = requests.get(url, params=params, headers=headers, verify=False)
print('Код запроса:', response.status_code)
response.json()

In [ ]:
l = []
for el in response.json()['subjects']:
    try:
        l.append({'subjectId' : el['subjectId'],'name' : el['name'], 'parentId' : el['parentId']})
    except Exception as e:
        l.append({'subjectId' : el['subjectId'],'name' : el['name'], 'parentId' : 'нет'})

In [ ]:
df = pd.DataFrame(l)
print(df.shape)
df.to_excel('subject_full.xlsx')

In [ ]:
import itertools  

a = list(df['subjectId'].unique())
b = list(df['parentId'].unique())
c = list(itertools.product(a, b))

In [ ]:
import networkx as nx

G=nx.Graph()
G.add_edges_from(c)
nx.draw_spring(G, )
# plt.savefig("simple_path.png")
# plt.show()

In [ ]:
# import networkx as nx
import holoviews as hv
from holoviews import opts

In [ ]:
 ### визуализация графов
hv.extension('bokeh')

In [ ]:
# параметры для построения графов
%opts Graph (edge_line_width=0.5)

In [ ]:
%opts Graph (node_size=5)

In [ ]:
 %opts Graph [width=800, height=800]

In [ ]:
df = pd.read_excel('subject.xlsx')
df.head()

In [ ]:
print(df.shape)
df = df[df['parentId'] != 'нет']
print(df.shape)

In [ ]:
gr = nx.from_pandas_edgelist(df, 'subjectId', 'parentId')

In [ ]:
gr

In [ ]:
hv.Graph.from_networkx(gr, nx.layout.spring_layout)

## Получение тематики источников

In [ ]:
api_irr = 'api/v1/sources/topics'
url = main_url + api_irr
print(url)

headers = {
    'Authorization': 'Bearer ' + accessToken
}

response = requests.get(url, headers=headers, verify=False)
print('Код запроса:', response.status_code)
response.json()

# Cценарии

## Загрузка архива публикаций

### Поиск по юридическим лицам

In [7]:
## Загружаем данные от Михеевой
data_b =pd.read_excel(r'Мониторинг СКАН Интерфакс (Белова).xlsx',
                         converters={'inn':str})
print(data_b.shape)
# собираем словарь
inn_list = list(data_b['inn'].unique())
companiesToSearch = []
for inn in inn_list:
    if len(inn) == 10:
        companiesToSearch.append({"inn":inn, "documentsCount": 100000})

(2071, 1)


In [8]:
## Загрузка архива публикаций
api_find_comp = 'api/v1/search/findByCompaniesList'
url = main_url + api_find_comp

print(url)

headers = {
    'Authorization': 'Bearer ' + accessToken
}

params = {
  "companiesToSearch": companiesToSearch,
  "period": {
    "startDate": "2020-03-01T00:00:00",
    "endDate": "2020-04-01T23:59:59"
  },
  "mentionContext": "any",
  "similarMode": "similar",
  "sortType": "newFirst",
  "dateSearchType": "publicationDate"
}

response = requests.post(url, json=params, headers=headers, verify=False)
print('Код запроса:', response.status_code)
res = response.json()

https://gateway.scan-interfax.ru/api/v1/search/findByCompaniesList
Код запроса: 200


In [10]:
len(res['companiesSearchResult'])

1959

In [11]:
df_list = []
for elem in res['companiesSearchResult']:
    mes_list = []
    for doc in elem['documentIds']:
        mes_list.append(doc)
    try:
        df_list.append({'company': elem['company']['inn'], 'queryCode':elem['queryCode'], 'totalCount': elem['totalCount'], 'documentIds':mes_list})
    except Exception as e:
        continue
        
result = pd.DataFrame(df_list)
result['len_documentIds'] = result['documentIds'].apply(lambda x: len(x))
result.head()

,company,documentIds,queryCode,totalCount,len_documentIds
0,6678034985,[1:VdC2GtGBUtGbRwsY0YBJZ+KAukjRmlfQvSfQnXXRj8K...,813811ece60d474af4de7b29b682d3e0,6,6
1,5406681543,[],af8bd3cfbcad886193798ab7116ad411,0,0
2,7814566648,[1:NNCYMNGa0JLQntGEwq1e0ZpwXOKAulBzT9CQD1k8U9C...,d2c8a496e69f0bf1b25ab969451a6978,2,2
3,7805555551,[],17f855487305a3f625a8853d3b087779,0,0
4,6163099251,[1:wrHRiSDRktCD0YDCp9CUQtCj0LdVH9GR0YfQhSfigJ1...,3c60e3760a33e96e3c2b1d85aed34699,4,4


In [12]:
result.to_excel('inn_res.xlsx')

In [13]:
result = result[result['len_documentIds']>0]
print(result.shape)
result.head()

(10, 5)


,company,documentIds,queryCode,totalCount,len_documentIds
0,6678034985,[1:VdC2GtGBUtGbRwsY0YBJZ+KAukjRmlfQvSfQnXXRj8K...,813811ece60d474af4de7b29b682d3e0,6,6
2,7814566648,[1:NNCYMNGa0JLQntGEwq1e0ZpwXOKAulBzT9CQD1k8U9C...,d2c8a496e69f0bf1b25ab969451a6978,2,2
4,6163099251,[1:wrHRiSDRktCD0YDCp9CUQtCj0LdVH9GR0YfQhSfigJ1...,3c60e3760a33e96e3c2b1d85aed34699,4,4
8,7811180727,[1:RRcOJdCgaMKg0JZ2wqTRg9GNPCDQhAhw0K9kFzUtUhT...,6fbb011cf4de4f418fb02c23d6c4427a,25,25
9,7814505170,[1:4oCd4oCUDOKAum8TaX9t0Y8c0LDRmdCXPNGLenTRm3E...,aa68b6a2d8391d7df957ca1179559488,6,6


### Получение текстов найденных публикаций

In [14]:
api_find_doc = 'api/v1/search/getDocuments'
url = main_url + api_find_doc

print(url)

headers = {
    'Authorization': 'Bearer ' + accessToken
}

con_df = []
for k, v in result.iterrows():
    params = {
      "queryCode": v['queryCode'],
      "ids": v['documentIds'],
      "attributes": [
        "content",  "objects"
      ],
      "options": [
        "inlineImages"
      ]
    }
    
    try:
        response = requests.post(url, json=params, headers=headers, verify=False)
        print('Код запроса:', response.status_code)
        docs = response.json()
        doc_list = []
        for elem in docs['documents']:
            doc_list.append({'documentId': elem['documentId'],
        #                     'dedupClusterId': elem['dedupClusterId'],
                             'date': elem['date'],
                             'loadingDate': elem['loadingDate'],
                             'title': elem['title'],
                             'author': elem['author'],
                             'sourceId': elem['sourceId'],
                             'sourceName': elem['sourceName'],
                             'url': elem['url'],
                             'fullTextUnavailable': elem['fullTextUnavailable'],
                             'content': elem['content'],
                             'objects': elem['objects'],
                            'queryCode':v['queryCode']})
        news = pd.DataFrame(doc_list)
        con_df.append(news)
    except Exception as e:
        print(e)
        continue
        
news = pd.concat(con_df)

https://gateway.scan-interfax.ru/api/v1/search/getDocuments
Код запроса: 200
Код запроса: 200
Код запроса: 200
Код запроса: 200
Код запроса: 200
Код запроса: 200
Код запроса: 200
Код запроса: 200
Код запроса: 200
Код запроса: 200


-----------------------

In [ ]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

In [ ]:
news['title'] = news['title'].apply(lambda x: cleanhtml(x))
news['content'] = news['content'].apply(lambda x: cleanhtml(x))
news.head()

In [ ]:
company = []
company_b = []
industrialSector = []
industrialSector_b = []
region = []
region_b = []
eventType = []
eventType_b = []
for o in news['objects']:
    for ob in o:
        if ob['type'] == 'company':
            company.append(ob['name'])
        elif ob['type'] == 'industrialSector':
            industrialSector.append(ob['name'])
        elif ob['type'] == 'region':
            region.append(ob['name'])
        elif ob['type'] == 'eventType':
            eventType.append(ob['name'])
    company_b.append(set(company))
    industrialSector_b.append(set(industrialSector))
    region_b.append(set(region))
    eventType_b.append(set(eventType))
    
news['company'] = company_b
news['industrialSector'] = industrialSector_b
news['region'] = region_b
news['eventType'] = eventType_b

In [ ]:
news = news[['queryCode', 'documentId', 'date', 'title', 'content', 'sourceName','url', 'company', 'industrialSector', 'region', 'eventType']]
news.head()

In [ ]:
final_table = news.merge(result, on='queryCode', how='inner')
final_table.head()

In [ ]:
final_table = final_table[['company_y', 'date', 'title', 'content', 'sourceName', 'url', 'industrialSector', 'region', 'eventType', 'totalCount']]
final_table.head()

In [ ]:
negative = pd.read_excel(r'source.xlsx')
negative = set(negative['name'].unique())
negative

In [ ]:
def real_neg(x):
    if len(x & negative) > 0:
        return True
    else:
        return False

In [ ]:
print(final_table.shape)
answer = final_table[final_table['eventType'].apply(real_neg)]
print(answer.shape)

In [ ]:
answer.to_excel('monitoring_scan.xlsx')